In [1]:
from in_scope_filter.in_scope_filter import In_Scope_Filter
from exp_rot_nets.expand_network import Expand_Network
from exp_rot_nets.rollout_network import Rollout_Network
from reaction import remove_mapped_num, single_step_synthesis, Reaction
import pandas as pd
from rdkit import Chem
import numpy as np
from sklearn.preprocessing import LabelEncoder
from MCTS.mcts_node import MCTSNode, State
from MCTS.mcts_policy import MCTS_Policy_old

/home/stein/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from hyperparams import Hyperparams as hp

In [3]:
hp.max_game_length

25

In [4]:
isf = In_Scope_Filter(model_path = "in_scope_filter/in_scope_filter_model/in_scope_filter.h5")

/home/stein/anaconda3/lib/python3.6/site-packages/keras/legacy/layers.py:198: UserWarning: The `Highway` layer is deprecated and will be removed after 06/2017.
  warnings.warn('The `Highway` layer is deprecated '
/home/stein/anaconda3/lib/python3.6/site-packages/keras/engine/saving.py:305: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [5]:
exp = Expand_Network(n_classes=10, n_feats=10000, model_path="exp_rot_nets/expnet_model/expnet.h5")

/home/stein/anaconda3/lib/python3.6/site-packages/keras/legacy/layers.py:198: UserWarning: The `Highway` layer is deprecated and will be removed after 06/2017.
  warnings.warn('The `Highway` layer is deprecated '
/home/stein/anaconda3/lib/python3.6/site-packages/keras/engine/saving.py:305: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [6]:
rot = Rollout_Network(n_classes=10, n_feats=10000, model_path="exp_rot_nets/rotnet_model/rotnet.h5")

/home/stein/anaconda3/lib/python3.6/site-packages/keras/engine/saving.py:305: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [7]:
exp_indexes = np.load("exp_rot_nets/expnet_model/top_10000_indexes.npy")
rot_indexes = np.load("exp_rot_nets/rotnet_model/top_10000_indexes.npy")

In [8]:
from utils import get_product_fingerprint

In [9]:
smarts = np.load("useful_smarts_pos_0.npy")

In [10]:
lbl2 = LabelEncoder()
lbl2.classes_ = np.load("exp_rot_nets/expnet_model/expnet_relabel.npy")
lbl1 = LabelEncoder()
lbl1.classes_ = np.load("exp_rot_nets/useful_data/expand_rule_label_encoder.npy")

In [11]:
product_smile1 = smarts[0].split(">>")[-1]
product_smile2 = smarts[1].split(">>")[-1]
mol1 = Chem.MolFromSmiles(product_smile1)
mol2 = Chem.MolFromSmiles(product_smile2)

In [12]:
init_state = State([product_smile1])

In [13]:
Searcher = MCTS_Policy_old(exp, exp, isf,lbl1, lbl2, exp_indexes, exp_indexes, num_readouts=10, verbosity=0)

In [14]:
Searcher.initialize(init_state)

In [15]:
Searcher.root

<MCTSNode move=(), N=0.0

In [16]:
first_node = Searcher.root.select_leaf(lbl1=lbl1, lbl2=lbl2, mol_index=0)

In [17]:
first_node.state.mols

['CCC[C@@]1(C[C@H](NC2=C1C=C(C=C2)C(N)=N)C1=CC=CC(=C1)C1=C(C=C(C=C1)C(O)=O)C(O)=O)C1=CC=CC=C1']

In [18]:
build_block_mols = ["a", 'b', 'c']

In [19]:
unsolved_mols, unsolved_indexes = first_node.state.get_unsolved_mols_and_indexes(build_block_mols)

In [20]:
for i, (mol, ind) in enumerate(zip(unsolved_mols, unsolved_indexes)):
    print(i, mol, ind)

0 CCC[C@@]1(C[C@H](NC2=C1C=C(C=C2)C(N)=N)C1=CC=CC(=C1)C1=C(C=C(C=C1)C(O)=O)C(O)=O)C1=CC=CC=C1 0


In [21]:
unsolved_mols

['CCC[C@@]1(C[C@H](NC2=C1C=C(C=C2)C(N)=N)C1=CC=CC(=C1)C1=C(C=C(C=C1)C(O)=O)C(O)=O)C1=CC=CC=C1']

In [22]:
unsolved_indexes

[0]

In [23]:
prob = exp.run(first_node.state, unsolved_indexes=unsolved_indexes, feat_indexes=exp_indexes)

In [24]:
prob

array([[7.178306e-06, 7.178306e-06, 7.178306e-06, ..., 7.178306e-06,
        7.178306e-06, 7.178306e-06]], dtype=float32)

In [25]:
prob.squeeze()

array([7.178306e-06, 7.178306e-06, 7.178306e-06, ..., 7.178306e-06,
       7.178306e-06, 7.178306e-06], dtype=float32)

In [26]:
first_node.incorporate_results(prob.squeeze(), build_block_mols=build_block_mols)

In [23]:
Searcher.root.N

0.0

In [24]:
kk = np.array([0,0,0,0])

In [25]:
kk.dtype = np.bool

In [26]:
kk

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False])

In [27]:
build_block_mols

['a', 'b', 'c']

In [23]:
readouts= 4
while True:
    current_readouts = Searcher.root.N
    # we want to do "X additional readouts", rather than "up to X readouts".
    while Searcher.root.N < current_readouts + readouts and (not Searcher.root.is_done(build_block_mols=build_block_mols)):
        Searcher.mcts_policy(build_block_mols=build_block_mols)
    if not Searcher.root.is_done(build_block_mols=build_block_mols):
        transformation = Searcher.pick_transformation()
        transformation = Searcher.decoding(transformation)
        Searcher.exec_transform(transformation)
    print("*"*100)
    if Searcher.root.is_done(build_block_mols=build_block_mols):
        Searcher.set_result(Searcher.root.state.is_solved(build_block_mols=build_block_mols))
        break

/home/stein/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/stein/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


parents [<MCTSNode move=(), N=0.0]
unsolved_mols ['CCC[C@@]1(C[C@H](NC2=C1C=C(C=C2)C(N)=N)C1=CC=CC(=C1)C1=C(C=C(C=C1)C(O)=O)C(O)=O)C1=CC=CC=C1']
leaf <MCTSNode move=('[Cl&H1:1]>>[O&H2:1]',), N=0.0


/home/stein/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/stein/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/stein/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/stein/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambig

leaf <MCTSNode move=('[Cl&H1:1]>>[O&H2:1]',), N=0.0
1


/home/stein/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/stein/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


...

/home/stein/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/stein/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


leaf <MCTSNode move=('[Cl&H1:1]>>[O&H2:1]',), N=0.0
24


/home/stein/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/stein/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/stein/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/stein/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambig

leaf <MCTSNode move=('[Cl&H1:1]>>[O&H2:1]',), N=0.0
25


AssertionError: 

In [24]:
Searcher.root.children

{'[Cl&H1:1]>>[O&H2:1]': <MCTSNode move=('[Cl&H1:1]>>[O&H2:1]',), N=1.0,
 '[C:1]=[O:2]>>[C&H2:1]': <MCTSNode move=('[C:1]=[O:2]>>[C&H2:1]',), N=1.0,
 '[S&H1:1].[S&H1:2]>>[S:1]-,:[S:2]': <MCTSNode move=('[S&H1:1].[S&H1:2]>>[S:1]-,:[S:2]',), N=1.0,
 '[C:2]-,:[N:1]>>[N&H1:1]': <MCTSNode move=('[C:2]-,:[N:1]>>[N&H1:1]',), N=1.0,
 '[C&H1:1].[C&H1:2]>>[C:1]-,:[C:2]': <MCTSNode move=('[C&H1:1].[C&H1:2]>>[C:1]-,:[C:2]',), N=0.0,
 '[O&H2:1]>>[O&H2:1]': <MCTSNode move=('[O&H2:1]>>[O&H2:1]',), N=0.0,
 '[S:1]>>[S:1]=[O:2]': <MCTSNode move=('[S:1]>>[S:1]=[O:2]',), N=0.0,
 '[C:2]-,:[N&H1:1]>>[N&H2:1]': <MCTSNode move=('[C:2]-,:[N&H1:1]>>[N&H2:1]',), N=0.0,
 '[C&H3:3]-,:[O:1].[O:2]-,:[C&H3:4]>>[O&H1:1].[O&H1:2]': <MCTSNode move=('[C&H3:3]-,:[O:1].[O:2]-,:[C&H3:4]>>[O&H1:1].[O&H1:2]',), N=0.0,
 '[S:1]=[O:2]>>[S:1]': <MCTSNode move=('[S:1]=[O:2]>>[S:1]',), N=0.0,
 '[c&H1:1]>>[c:1]-,:[N&+:2]': <MCTSNode move=('[c&H1:1]>>[c:1]-,:[N&+:2]',), N=0.0,
 '[C:1]-,:[C:2]>>[C&H1:1]': <MCTSNode move=('[C:1]-,:[C:2]

In [69]:
Searcher.root.children["[Cl&H1:1]>>[O&H2:1]"].children

{'[Cl&H1:1]>>[O&H2:1]': <MCTSNode move=('[Cl&H1:1]>>[O&H2:1]',), N=1.0,
 '[C:1]=[O:2]>>[C&H2:1]': <MCTSNode move=('[C:1]=[O:2]>>[C&H2:1]',), N=0.0,
 '[S&H1:1].[S&H1:2]>>[S:1]-,:[S:2]': <MCTSNode move=('[S&H1:1].[S&H1:2]>>[S:1]-,:[S:2]',), N=0.0,
 '[C:2]-,:[N:1]>>[N&H1:1]': <MCTSNode move=('[C:2]-,:[N:1]>>[N&H1:1]',), N=0.0,
 '[C&H1:1].[C&H1:2]>>[C:1]-,:[C:2]': <MCTSNode move=('[C&H1:1].[C&H1:2]>>[C:1]-,:[C:2]',), N=0.0,
 '[O&H2:1]>>[O&H2:1]': <MCTSNode move=('[O&H2:1]>>[O&H2:1]',), N=0.0,
 '[S:1]>>[S:1]=[O:2]': <MCTSNode move=('[S:1]>>[S:1]=[O:2]',), N=0.0,
 '[C:2]-,:[N&H1:1]>>[N&H2:1]': <MCTSNode move=('[C:2]-,:[N&H1:1]>>[N&H2:1]',), N=0.0,
 '[C&H3:3]-,:[O:1].[O:2]-,:[C&H3:4]>>[O&H1:1].[O&H1:2]': <MCTSNode move=('[C&H3:3]-,:[O:1].[O:2]-,:[C&H3:4]>>[O&H1:1].[O&H1:2]',), N=0.0,
 '[S:1]=[O:2]>>[S:1]': <MCTSNode move=('[S:1]=[O:2]>>[S:1]',), N=0.0,
 '[c&H1:1]>>[c:1]-,:[N&+:2]': <MCTSNode move=('[c&H1:1]>>[c:1]-,:[N&+:2]',), N=0.0,
 '[C:1]-,:[C:2]>>[C&H1:1]': <MCTSNode move=('[C:1]-,:[C:2]

In [73]:
Searcher.root.children["[C:1]=[O:2]>>[C&H2:1]"].children["[c:1]-,:[Br:2]>>[c&H1:1]"].state.mols

['CCCC1(CC(Nc2ccc(cc21)C(=N)N)c1cccc(c1)-c1ccc(cc1C(=O)O)CO)c1ccccc1']

In [38]:
Searcher.result_string

In [38]:
Searcher.set_result(Searcher.root.state.is_solved(build_block_mols=build_block_mols))

In [39]:
Searcher.root.state.is_solved(build_block_mols=build_block_mols)

False

In [44]:
not Searcher.root.is_done(build_block_mols=build_block_mols)

False

In [33]:
Searcher.root.children

{}